In [1]:
!pip install pandas numpy scikit-learn nltk sentence-transformers


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
!pip install PyPDF2 langchain

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 3.9 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 3.9 MB/s eta 0:00:01
   ------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import PyPDF2
import re
import os
import pandas as pd # pandas is useful for organizing extracted data later

# --- Configuration ---
# Set the directory where your PDF books are located
# Make sure your PDF files are in a 'books' subfolder within your ChatBotProject folder.
pdf_dir = 'books'

# List of PDF file names you want to process
pdf_files = [
    'C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf',
    'C:/Users/Ayush Jindal/Downloads/Wallach.pdf',
    'C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf'
]

# --- Text Extraction Function ---
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file page by page.
    Includes error handling for missing files or extraction issues.
    """
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            num_pages = len(reader.pages)
            print(f"  Extracting text from {num_pages} pages...")
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                try:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text
                except Exception as page_e:
                    print(f"    Warning: Could not extract text from page {page_num + 1} of {os.path.basename(pdf_path)}: {page_e}")
                    text += "[PAGE_TEXT_EXTRACTION_ERROR]"
        print(f"Successfully extracted raw text from: {os.path.basename(pdf_path)}")
    except FileNotFoundError:
        print(f"Error: PDF file not found at '{pdf_path}'. Please check the path and filename.")
    except Exception as e:
        print(f"Error opening or reading PDF '{pdf_path}': {e}")
    return text

# --- Initial Text Cleaning Function ---
def clean_initial_pdf_text(text):
    """
    Performs initial cleaning on raw extracted PDF text.
    Removes common PDF artifacts and standardizes formatting.
    """
    # 1. Remove source tags like or 

    # 2. Remove common PDF rendering artifacts like (cid:xx)
    text = re.sub(r'\(cid:\d+\)', '', text)

    # 3. Remove specific repeating headers/footers based on observation of your PDFs
    text = re.sub(r'Routine Blood\n Results\n Explained\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Routine Blood Results Explained', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in\n Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Blood Results in Clinical Practice', '', text, flags=re.IGNORECASE)
    text = re.sub(r'A practical guide\n to interpreting blood\n test results', '', text, flags=re.IGNORECASE)

    # 4. Remove page numbers. This is heuristic and targets various page number patterns.
    text = re.sub(r'^\s*\d+\s*(\n|$)', '\n', text, flags=re.MULTILINE)
    text = re.sub(r'(\n|^)\s*\d+\s*(\n|$)', '\n', text)
    text = re.sub(r'Page \d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Chapter \d+\s*\d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^\s*\d{1,3}\s*$', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\s*(viii|ix|x|xi|xii|xiii|xiv|xv|xvi|xvii|xviii|xix|xx)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # 5. Remove TOC-like entries that remain, or prefaces/abbreviations
    text = re.sub(r'^\s*(Contents|Preface|Abbreviations|Introduction|Part \d+|Chapter \d+).*?\b(?:vii|viii|ix|x|xi|xii|\d+)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # 6. Standardize whitespace: Replace multiple whitespaces (including newlines) with single space
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'([.?!])\s*(?=[A-Z0-9]|$)', r'\1\n\n', text) # Add double newline after sentences
    text = re.sub(r'\n{3,}', '\n\n', text) # Reduce excessive newlines

    # 7. Remove hyphenation from words split across lines
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)

    # 8. Clean specific table artifacts from contents that might be extracted oddly
    text = re.sub(r'"([^"]+)"\s*,\s*"([^"]+)"', r'\1 \2', text)
    text = re.sub(r'Objectives and Scope\n', '', text)
    text = re.sub(r'Table \d+\.\d+:.*?\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'The following table:', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Mode LastWriteTime Length ---- ------------- ------', '', text)

    return text

# --- Main Processing Loop ---
cleaned_texts_by_book = {}

for pdf_file_name in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file_name)
    print(f"\n--- Processing: {pdf_file_name} ---")

    raw_text = extract_text_from_pdf(pdf_path)
    if raw_text:
        cleaned_text = clean_initial_pdf_text(raw_text)
        cleaned_texts_by_book[pdf_file_name] = cleaned_text
        print(f"Initial cleaning complete for {pdf_file_name}. Cleaned length: {len(cleaned_text)} characters.")
    else:
        print(f"Skipping {pdf_file_name} due to extraction error or empty content.")

print("\n--- Initial Text Cleaning Summary ---")
for book_name, text_content in cleaned_texts_by_book.items():
    print(f"Book: {book_name[:35]}... | Length: {len(text_content)} chars")
    print("Sample (first 1000 chars):\n", text_content[:1000])
    print("-" * 50)


--- Processing: C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf ---
  Extracting text from 172 pages...
Successfully extracted raw text from: ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf
Initial cleaning complete for C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf. Cleaned length: 264579 characters.

--- Processing: C:/Users/Ayush Jindal/Downloads/Wallach.pdf ---
  Extracting text from 1884 pages...
Successfully extracted raw text from: Wallach.pdf
Initial cleaning complete for C:/Users/Ayush Jindal/Downloads/Wallach.pdf. Cleaned length: 3245859 characters.

--- Processing: C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf ---
  Extracting text from 119 pages...
Successfully extracted raw text f

In [18]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Ayush
[nltk_data]     Jindal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to C:\Users\Ayush
[nltk_data]     Jindal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\Users\Ayush
[nltk_data]     Jindal\AppData\Roaming\nltk_data...


True

In [20]:
import re
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize # Ensure word_tokenize is imported if needed for preprocess_text_for_chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Re-define or ensure cleaned_texts_by_book is available from Step 1 (it should be if previous cell ran)
# This block is here ONLY as a fallback if the previous cell wasn't run successfully (which it now has)
# So, in your live notebook, cleaned_texts_by_book will already be populated.
if 'cleaned_texts_by_book' not in locals() or not cleaned_texts_by_book:
    print("Error: 'cleaned_texts_by_book' dictionary not found or empty. Please run Step 1 successfully first.")
    # Fallback to dummy data for demonstration if previous step failed
    # This dummy data has been created to mimic a slightly more realistic output for testing chunking
    cleaned_texts_by_book = {
        'ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf': """INTRODUCTION: ROUTINE BLOOD RESULTS EXPLAINED. "...it is estimated that the data received by clinicians from Medical Laboratories constitutes 70-80% of the information they rely on to make major medical decisions" The Biomedical Scientist 2005:49; page 38. Blood tests are important as they provide three times as much information as do all other sources (history, examination, symptoms, imaging etc.) combined. Fortunately, the vast majority of routine blood tests, certainly in routine, emergency, and critical care medicine, fall easily into defined groups haematology (with blood transfusion), immunology, and biochemistry, together often described as Blood Science. The layout of the volume will therefore follow this pattern. Each of the major sections breaks down into individual chapters and concludes with a dedicated example. Knowledge is nothing without practice. Therefore, the book will conclude with case studies designed to help the practitioner. These cases will look at both primary and secondary care. What is done where? In some Pathology Departments, certain tests are done in the Haematology Laboratory, whilst in other Hospitals the same test may be performed in the Biochemistry or Immunology Laboratory, or perhaps a Department of Blood Science. Examples of this include iron studies, immunoglobulins, C-reactive protein (CRP), and testing for vitamin B12. These latter tests are done on serum obtained from whole blood that has not been anticoagulated, but some tests are performed on blood where clotting has been prevented by anticoagulants. The reader is referred to their own Pathology Service for the correct tube for the test and the destination of these requests. Overall, our colleagues in the Pathology Department, regardless of discipline, would far rather set the position clear in a phone call than go through the bother of phoning back that a fresh sample in the correct tube must be obtained. If in doubt - PHONE! A note on units. In the real world, of course, results are almost always described as the numbers themselves (e.g., a haemoglobin of 125 or a cholesterol of 5) instead of the more correct way its unit (i.e., 125 g/L and 8 mmol/L). This shorthand is generally accepted, and generally makes life considerable easier. It matters not so much that the correct unit of the average size of a red blood cell is described fully, for example, as 112 fL, or in shorthand simply as 112, but it does matter that the particular cell is much larger than can be expected in health, implying come pathological condition. The reference ranges. In defining ill-health, we generally use good health as a comparator. Thus, a healthy person can be expected to have a certain blood result profile. However, sometimes these values are not well established and are subject to variation. Furthermore, there are many normal (healthy) people whose blood result may not be in the expected range of values this does not necessarily mean they are ill. Therefore, the concept of being 'normal' may as well be 'desirable'. So, you could use a 'reference' range, or perhaps a 'target' range. However, for the purposes of this volume, the reference range will be cited. Haematology, biochemistry, and immunology are very quantitative sciences. Consequently, the reference range is important. The precise definition of the reference range in use at a particular hospital is crucial and may not be transferable to another hospital. This may be because of small differences in the technical manner in which tests are derived. Furthermore, reference ranges may well (or actually should) reflect the local population that the hospital serves, and this is important as different catchment populations may well vary considerably, especially in race and ethnicity.
        """,
        'Wallach.pdf': """Wallach’s Interpretation of Diagnostic Tests. L. V. Rao, PhD, FAACC Professor of Pathology. University of Massachusetts Medical School. Worcester, Massachusetts. Executive Director, Science Quest Diagnostics, North Region. Marlborough, Massachusetts. L. Michael Snyder, MD Professor. Department of Medicine and Pathology. University of Massachusetts Medical School. UMass Memorial Medical Center. Worcester, Massachusetts. Academic Associate Quest Diagnostics MA, LLC. Marlborough, Massachusetts. Eleventh Edition Copyright © 2021 Wolters Kluwer. All rights reserved. The Kidney. Renal Function. Kidneys filter blood. Urea and creatinine are waste products. GFR measures kidney function. High creatinine means poor kidney function. Kidney failure is serious. Dialysis is a treatment. Electrolytes. Sodium and potassium are electrolytes. They control water balance. High potassium is dangerous. Low sodium can mean dehydration. Diabetes. Diabetes is high blood sugar. Insulin controls sugar. Type 1 and Type 2 diabetes exist. HbA1c measures long-term control. Hypoglycemia is low blood sugar. Complications include neuropathy. Foot problems are common. Cholesterol and heart disease. Lipids are fats. HDL is good cholesterol. LDL is bad cholesterol. Statins lower cholesterol. The body's organs and functions are incredibly complicated and interlinked. The book will therefore outline such links in terms of 'family groups'. For example, the kidney is responsible for supporting the production of red blood cells; when looking at the full blood count (FBC), it may therefore be helpful to look at the renal or kidney function. These connections are highlighted in the text by the LINK symbol. Blood tests seldom provide a diagnosis on their own; they are best used in conjunction with case history, X-rays, scans and other reports by allied health professionals. Given that reference ranges are specific to the machine which analyses the blood in a particular healthcare setting, no formal ranges are presented in this book. You must only use the range presented alongside the result or the range approved by your local healthcare setting. The values and interpretations used in the book are based on current national guidance. Readers should always seek definitive local approved guidance on diagnosis, treatment, additional blood tests to request and file, and repeat timeframes.
        """,
        'Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf': """Blood Results in Clinical Practice. A practical guide to interpreting blood test results. Graham Basten. 2nd edition. Dr Graham Basten ISBN: 978-1-910451-16-8. First published 2013. This revised and updated edition published 2019. All rights reserved. The second edition of this book continues to use storytelling to aid understanding, and also introduces a new and unique 10-point system to help explain blood results. The use of storytelling has been significantly improved and refined following several years of feedback on the first edition. The purpose of the book has also evolved. Given that so many protocols and decisions can now be found online, and many NHS trusts and private providers produce similar flow charts, the character of the book has been adjusted: it is less formal than a biochemistry textbook while still containing more narrative than an online protocol. It provides an excellent, accessible introduction to blood tests and what they mean. It also enables advanced practitioners to reflect on and improve their practice, and includes new and updated sections of relevance to physiotherapists, paramedics, pharmacists and advanced nurse practitioners. The main audiences for the book are: 1) Undergraduate or postgraduate healthcare students 2) Healthcare professionals who need an essential handbook for quick reference or as an accessible text for a new area of practice (for example, new MSK First Contact or paramedic practitioners in primary and community care) 3) Patients, clients, friends and relatives who may wish to know more about what a particular blood test means. Throughout the book, storytelling is used. For instance, we will talk about 'fire engines' and 'police cars' to help explain what are essentially abstract concepts. When a storytelling device is being used, this will be flagged up in the text as 'storytelling'. The human body's organs and functions are incredibly complicated and interlinked. The book will therefore outline such links in terms of 'family groups'. For example, the kidney is responsible for supporting the production of red blood cells; when looking at the full blood count (FBC), it may therefore be helpful to look at the renal or kidney function. These connections are highlighted in the text by the LINK symbol. Blood tests seldom provide a diagnosis on their own; they are best used in conjunction with case history, X-rays, scans and other reports by allied health professionals. Given that reference ranges are specific to the machine which analyses the blood in a particular healthcare setting, no formal ranges are presented in this book. You must only use the range presented alongside the result or the range approved by your local healthcare setting. The values and interpretations used in the book are based on current national guidance. Readers should always seek definitive local approved guidance on diagnosis, treatment, additional blood tests to request and file, and repeat timeframes.
        """
    }
    # Apply basic cleaning to dummy text to match expected input from Step 1
    for book_name in cleaned_texts_by_book:
        cleaned_texts_by_book[book_name] = re.sub(r'\s+', ' ', cleaned_texts_by_book[book_name]).strip()
        cleaned_texts_by_book[book_name] = re.sub(r'\n{2,}', '\n\n', cleaned_texts_by_book[book_name])


# --- Advanced Text Preprocessing and Chunking Function ---
def clean_and_chunk_text(book_text, book_title, chunk_size=500, chunk_overlap=100):
    """
    Applies further cleaning and splits text into smaller, overlapping chunks suitable for LLMs.
    """
    # Apply light preprocessing
    # Remove chapter/page number patterns from headers/footers that might remain
    processed_text = re.sub(r'Chapter \d+.*?(\d+|vii|ix|x|xi)\s*$', '', book_text, flags=re.MULTILINE)
    processed_text = re.sub(r'(\n|^)\s*(Preface|Abbreviations|Introduction|Part \d+|Contents)\s*$', '', processed_text, flags=re.MULTILINE | re.IGNORECASE)
    processed_text = re.sub(r'\s+', ' ', processed_text).strip() # Standardize whitespace
    processed_text = re.sub(r'\n{3,}', '\n\n', processed_text) # Reduce excessive newlines
    processed_text = re.sub(r'([.?!])\s*(?=[A-Z0-9]|$)', r'\1\n\n', processed_text) # Add double newline after sentences
    processed_text = re.sub(r'\n{3,}', '\n\n', processed_text) # Re-reduce excessive newlines after sentence splitting

    # Initialize text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""] # Prioritize splitting by larger units first
    )

    # Split the document into chunks
    chunks = text_splitter.split_text(processed_text)
    
    # Add metadata to each chunk
    chunks_with_metadata = []
    for i, chunk in enumerate(chunks):
        chunks_with_metadata.append({
            'book_name': book_title,
            'chunk_id': f"{os.path.splitext(book_title)[0].replace(' ', '_').replace('.', '').replace('__', '_')}_chunk_{i:04d}", # Cleaned ID for potential vector DB use
            'chunk_text': chunk,
            'chunk_length': len(chunk)
            # 'start_char': book_text.find(chunk) # Approximate start char - may not be accurate after extensive cleaning
        })
    return chunks_with_metadata

# --- Main Chunking Process ---
all_processed_chunks = []

for book_name, text_content in cleaned_texts_by_book.items():
    print(f"\n--- Chunking text for: {book_name} ---")
    chunks_for_book = clean_and_chunk_text(text_content, book_name, chunk_size=500, chunk_overlap=100) # Adjust chunk_size/overlap as needed
    all_processed_chunks.extend(chunks_for_book)
    print(f"Generated {len(chunks_for_book)} chunks for {book_name}.")

# Create a DataFrame of all chunks with their metadata
df_book_chunks = pd.DataFrame(all_processed_chunks)

print("\n--- Chunks DataFrame Head (from all books) ---")
print(df_book_chunks.head())
print(f"\nTotal number of chunks generated: {len(df_book_chunks)}")
print(f"Average chunk length: {df_book_chunks['chunk_length'].mean():.2f} characters.")

# Example: Display a few sample chunks for review
if not df_book_chunks.empty:
    print("\n--- Sample Chunks for Review ---")
    for i in range(min(3, len(df_book_chunks))): # Display first 3 chunks
        print(f"\nChunk ID: {df_book_chunks['chunk_id'].iloc[i]}")
        print(f"Book: {df_book_chunks['book_name'].iloc[i]}")
        print(f"Text ({df_book_chunks['chunk_length'].iloc[i]} chars):")
        print(df_book_chunks['chunk_text'].iloc[i])
        print("-" * 70)


--- Chunking text for: C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf ---
Generated 684 chunks for C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD RESULTS EXPLAINED - ANDREW BLANN (2022, CAMBRIDGE SCHOLARS PUB) (1).pdf.

--- Chunking text for: C:/Users/Ayush Jindal/Downloads/Wallach.pdf ---
Generated 8485 chunks for C:/Users/Ayush Jindal/Downloads/Wallach.pdf.

--- Chunking text for: C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf ---
Generated 485 chunks for C:/Users/Ayush Jindal/Downloads/Blood Results in Clinical Practice_ A practical guide to interpreting blood test results - Graham Basten (2019, M&K Update Ltd).pdf.

--- Chunks DataFrame Head (from all books) ---
                                           book_name  \
0  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   
1  C:/Users/Ayush Jindal/

In [21]:
!pip install torch sentence-transformers


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
from sentence_transformers import SentenceTransformer
import torch
import pandas as pd # Ensure pandas is imported if not already in the cell

# Ensure df_book_chunks is available from Step 2 (Advanced Cleaning and Semantic Chunking)
# If for some reason df_book_chunks is empty or not found, this block provides dummy data for demonstration.
if 'df_book_chunks' not in locals() or df_book_chunks.empty:
    print("Warning: 'df_book_chunks' DataFrame is empty or not found. Using dummy data for demonstration.")
    df_book_chunks = pd.DataFrame({
        'book_name': ['Dummy Book 1.pdf', 'Dummy Book 1.pdf', 'Dummy Book 2.pdf'],
        'chunk_id': ['dummy_book_1_chunk_0001', 'dummy_book_1_chunk_0002', 'dummy_book_2_chunk_0001'],
        'chunk_text': [
            'This chunk explains the full blood count (FBC) and its components like red blood cells and white blood cells.',
            'Another chunk discusses coagulation, D-dimers, and anticoagulants like warfarin and heparin for thrombosis.',
            'This section focuses on liver function tests, bilirubin, ALT, and AST, and their role in diagnosing jaundice.'
        ],
        'chunk_length': [120, 150, 130]
    })
    print("Dummy df_book_chunks created for demonstration.")


# Initialize a Sentence Transformer model
print("Loading Sentence Transformer model...")
# 'all-MiniLM-L6-v2' is a good balance of size and performance for general-purpose embeddings.
# This will download the model weights the first time you run this command, which may take a few minutes.
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded.")

# Generate embeddings for each chunk
print(f"\nGenerating embeddings for {len(df_book_chunks)} chunks. This may take some time for large datasets...")
# Convert the 'chunk_text' column to a list for the model's input
chunk_texts_list = df_book_chunks['chunk_text'].tolist()

# Generate embeddings. The model.encode method handles batching and GPU usage automatically if available.
# Set convert_to_tensor=False to get NumPy arrays directly if you don't need PyTorch tensors.
chunk_embeddings = model.encode(chunk_texts_list, show_progress_bar=True, convert_to_tensor=False)

print(f"Generated embeddings with shape: {chunk_embeddings.shape}")

# Add the embeddings as a new column to the DataFrame
# Storing NumPy arrays directly in a DataFrame column is efficient.
df_book_chunks['embedding'] = list(chunk_embeddings)

print("\n--- Final Chunks DataFrame Head with Embeddings ---")
print(df_book_chunks.head())
print(f"\nFinal DataFrame shape with embeddings: {df_book_chunks.shape}")

# You can now optionally save this vectorized data. Parquet is an excellent format for DataFrames
# with numerical array columns like embeddings, especially if you have many chunks.
# df_book_chunks.to_parquet('vectorized_medical_book_chunks.parquet', index=False)
# print("\nVectorized book chunks saved to 'vectorized_medical_book_chunks.parquet'")

# This `df_book_chunks` DataFrame, with its 'book_name', 'chunk_id', 'chunk_text', and 'embedding' columns,
# is the clean, vectorized data suitable for feeding into an LLM system. This data can be used for:
# - Storing in a Vector Database for Retrieval Augmented Generation (RAG).
# - Directly using for similarity search to find relevant information based on user queries.
# - Fine-tuning an LLM (though this is more complex and typically requires much more data).

C:\Users\Ayush Jindal\Desktop\ChatBotProject\chatbot_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Sentence Transformer model...


C:\Users\Ayush Jindal\Desktop\ChatBotProject\chatbot_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ayush Jindal\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is 

Model loaded.

Generating embeddings for 9654 chunks. This may take some time for large datasets...


Batches: 100%|█| 302/302 [05:49<00:00,  1.1


Generated embeddings with shape: (9654, 384)

--- Final Chunks DataFrame Head with Embeddings ---
                                           book_name  \
0  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   
1  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   
2  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   
3  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   
4  C:/Users/Ayush Jindal/Downloads/ROUTINE BLOOD ...   

                                            chunk_id  \
0  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   
1  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   
2  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   
3  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   
4  C:/Users/Ayush_Jindal/Downloads/ROUTINE_BLOOD_...   

                                          chunk_text  chunk_length  \
0  Routine Blood Results Explained Routine Blood ...           495   
1  book is available from the British Library Cop...           128   
2  No pa rt of thi

In [23]:
import pandas as pd # Ensure pandas is imported

# Assuming df_book_chunks is your DataFrame containing the chunks and embeddings
# from the previous "Convert Chunks to Vector Form" step.

try:
    df_book_chunks.to_parquet('vectorized_medical_book_chunks.parquet', index=False)
    print("\nVectorized data successfully saved as 'vectorized_medical_book_chunks.parquet'")
    print(f"File saved to: {os.path.join(os.getcwd(), 'vectorized_medical_book_chunks.parquet')}")
except Exception as e:
    print(f"Error saving to Parquet: {e}")
    print("Attempting to save as CSV as a fallback (embeddings will be stored as strings).")
    try:
        df_book_chunks.to_csv('vectorized_medical_book_chunks.csv', index=False)
        print("Vectorized data saved as 'vectorized_medical_book_chunks.csv'")
    except Exception as csv_e:
        print(f"Error saving to CSV: {csv_e}")

Error saving to Parquet: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.
Attempting to save as CSV as a fallback (embeddings will be stored as strings).
Vectorized data saved as 'vectorized_medical_book_chunks.csv'


In [24]:
!pip install pyarrow

   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   - -------------------------------------- 0.8/25.7 MB 7.6 MB/s eta 0:00:04
   -- ------------------------------------- 1.8/25.7 MB 4.5 MB/s eta 0:00:06
   ---- ----------------------------------- 2.6/25.7 MB 4.2 MB/s eta 0:00:06
   ----- ---------------------------------- 3.4/25.7 MB 4.1 MB/s eta 0:00:06
   ------ --------------------------------- 4.2/25.7 MB 4.0 MB/s eta 0:00:06
   ------- -------------------------------- 5.0/25.7 MB 4.0 MB/s eta 0:00:06
   -------- ------------------------------- 5.8/25.7 MB 4.0 MB/s eta 0:00:06
   ---------- ----------------------------- 6.6/25.7 MB 3.9 MB/s eta 0:00:05
   ----------- ---------------------------- 7.3/25.7 MB 3.9 MB/s eta 0:00:05
   ----------- ---------------------------- 7.6/25.7 MB 3.9 MB/s eta 0:00:05
   ------------- -------------------------- 8.7/25.7 MB 3.8 MB/s eta 0:00:05
   -------------- ------------------------- 9.2/25.7 MB 3.7 MB/s eta 0:00:05
   ---


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
# Assuming df_book_chunks is your DataFrame containing the chunks and embeddings
# from the previous "Convert Chunks to Vector Form" step.

try:
    df_book_chunks.to_parquet('vectorized_medical_book_chunks.parquet', index=False)
    print("\nVectorized data successfully saved as 'vectorized_medical_book_chunks.parquet'")
    print(f"File saved to: {os.path.join(os.getcwd(), 'vectorized_medical_book_chunks.parquet')}")
except Exception as e:
    print(f"Error saving to Parquet: {e}")
    print("Saving to CSV as a fallback failed or was not attempted due to prior success.")


Vectorized data successfully saved as 'vectorized_medical_book_chunks.parquet'
File saved to: C:\Users\Ayush Jindal\Desktop\ChatBotProject\chatbot_env\vectorized_medical_book_chunks.parquet
